# CME 241 Assignment 9

## Shaan Patel

### Question 1

In [10]:
from typing import Dict, Tuple
from rl.chapter9.order_book import DollarsAndShares, PriceSizePairs, OrderBook
from rl.distribution import Distribution
from rl.markov_process import MarkovRewardProcess, NonTerminal, State
from numpy.random import poisson
from scipy.stats import poisson as pois

In [ ]:
bids: PriceSizePairs = [DollarsAndShares(
        dollars=x,
        shares=poisson(100. - (100 - x) * 10)
    ) for x in range(100, 90, -1)]
asks: PriceSizePairs = [DollarsAndShares(
        dollars=x,
        shares=poisson(100. - (x - 105) * 10)
    ) for x in range(105, 115, 1)]

ob = OrderBook(descending_bids=bids, ascending_asks=asks)

class OBDynam(MarkovRewardProcess[OrderBook, DollarsAndShares]):
    def transition(self, state: OrderBook) -> Distribution[Tuple[OrderBook, DollarsAndShares]]:
        d: Dict[Tuple[OrderBook, DollarsAndShares], float] = {}
        bidshares = 0
        for i in range(len(state.descending_bids)):
            bidshares += state.descending_bids[i].shares
        askshares = 0
        for i in range(len(state.ascending_asks)):
            askshares += state.ascending_asks[i].shares
        
        poisson_distr = pois(state.mid_price)
        
        
        


### Question 2

Given that we have a temporary price impact of:

$$Q_t = P_t (1- \beta N_t - \theta X_t) $$

The optimal value function is:

$$V^*_t((P_t, R_t)) = \max_{N_t} \{N_t P_t(1 - \beta N_t - \theta X_t) + E[V^*_{t+1}((P_{t+1},R_{t+1}))]\} $$

$$V^*_{T-1}((P_{T-1}, R_{T-1})) = N_{T-1} P_{T-1}(1 - \beta N_{T-1} - \theta X_{T-1})$$

$$ = R_{T-1} P_{T-1}(1 - \beta R_{T-1} - \theta X_{T-1}) $$

Therefore we can infer $V^*_{T-2}((P_{T-2}, R_{T-2}))$ as:

$$\max_{N_{T-2}}\{N_{T-2}P_{T-2}(1 - \beta N_{T-2} - \theta X_{T-2}) + E[R_{T-1} P_{T-1}(1 - \beta R_{T-1} - \theta X_{T-1})]\} $$

$$\max_{N_{T-2}}\{N_{T-2}P_{T-2}(1 - \beta N_{T-2} - \theta X_{T-2}) $$
$$+ E[(R_{T-2} - N_{T-2}) P_{T-1}(1 - \beta (R_{T-2} - N_{T-2}) - \theta X_{T-1})]\} $$

This leads to:

$$ {N_{T-2}}N_{T-2}P_{T-2}(1 - \beta N_{T-2} - \theta X_{T-2}) \\
+ E[(R_{T-2} - N_{T-2}) (P_{T-2}e^{Z_{T-2}})(1 - \beta (R_{T-2} - N_{T-2}) - \theta (\rho X_{T-2} + \eta_{T-2}))] $$

The expected value can then clear by knowing the means of our random variables. By grouping like terms, we get:

$$ \max_{N_{T_2}} N_{T-2}P_{T-2}(1 - e^{\mu_Z + \frac{\sigma_Z^2}{2}}) - \beta N^2_{T-2}P_{T-2} (1 + e^{\mu_Z + \frac{\sigma_Z^2}{2}}) \\
 - \beta P_{T-2}e^{\mu_Z + \frac{\sigma_Z^2}{2}} (R^2_{T-2} - 2N_{T-2}R_{T-2}) \\
  - \theta N_{T-2}P_{T-2}X_{T-2}(1 - \rho e^{\mu_Z + \frac{\sigma_Z^2}{2}}) - \theta \rho  R_{T-2} P_{T-2} X_{T-2} e^{\mu_Z + \frac{\sigma_Z^2}{2}}$$